In [51]:
import pandas as pd
import ipaddress
from sklearn import preprocessing
import numpy as np
import itertools
import subprocess
import os
import sqlite3
from random import randint

In [68]:
def get_events_in_this(i_dfm, i_dir):
    out = []
    if i_dir == '0':
        ret_list = {'loader': 'dport_6.8000',\
                'cnc': 'dport_6.4567',\
                'http-ddos': 'dport_6.80',\
                'dns-rddos': 'dport_17.53',\
                'scan-out': 'dport_6.22'
               }
        local_dport = i_dfm['dport'].unique().tolist()
        
        #print(local_dport)
        
        for k,v in ret_list.items():
            if v in local_dport:
                out.append(k)       
    else:
        l_temp = i_dfm[((i_dfm['ext'].str.startswith('ext_172.16.'))|\
                        (i_dfm['ext'].str.startswith('ext_192.168.')))].shape[0]  
        if l_temp > 0:
            out.append('scan-in')
            out.append('login')
    print('Events->')
    print(out)
    return out

In [115]:
def get_masked_ip(i_ip, i_mask='16'):
    try:
        return str(ipaddress.IPv4Network(i_ip+"/"+i_mask, strict=False).network_address)
    except:
        return i_ip
    
def get_encoded_dataset(i_path, i_subnet=False, i_timeslot=0, \
                       output_mapping_file ='/tmp/mapping.txt',\
                       output_file_prefix = '/tmp/input'):
    df_in=pd.read_csv(i_path, sep=',', dtype=str)
    start_time = df_in['time'].astype(float).min() 
    df_in['time_diff'] = df_in['time'].astype(float) - start_time
    cols_to_delete = ['dir', 'label', 'time', 'time_diff']
    
    if i_subnet:
        print('[+] Subnet option is ENABLED.')
        df_in['iot'] = list(map(get_masked_ip, df_in['iot']))
        df_in['ext'] = list(map(get_masked_ip, df_in['ext']))
    else:
        print('[+] Subnet option is DISABLED.')
    
    cols = df_in.columns.tolist()
    data_col  = cols.copy()

    for a_col in cols_to_delete:
        data_col.remove(a_col)

    for col in data_col:
        df_in[col] = df_in[col].apply(lambda x: col+'_'+str(x))

    uniq = []
    for col in data_col:
        uniq+=df_in[col].unique().tolist()

    le = preprocessing.LabelEncoder()
    le.fit(uniq)

    le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
    
    with open(output_mapping_file, 'w') as file:
        for k,v in le_name_mapping.items():
            file.write("{} {}\n".format(k,v))

    
    if i_timeslot > 0:
        print('[+] Generating timeslotted entries.')
        ################# with time step ######################
        STEP = i_timeslot
        ts = 0.0
        count = 0    
        while True:
            df_entries_t = df_in[ (df_in['time_diff'] > ts) &  (df_in['time_diff'] <= ts+STEP)]
            if df_entries_t.shape[0] > 0:
                for flow_dir in ['0','1']:

                    print('Dir :{}, Timestep :{}'.format(flow_dir, count))
                    
                    df_entries = df_entries_t[df_entries_t['dir']==flow_dir][data_col]
                    
                    get_events_in_this(df_entries, flow_dir)
                    
                    df_entries.to_csv(output_file_prefix+"_dir_{}_step_{}_plain.dat".format(flow_dir,count),\
                                header=False, index=False, mode='w', sep=' ')
                    
                    bfr = pd.DataFrame()
                    
                    for cols in df_entries.columns.get_values():
                        bfr[cols] = df_entries[cols].map(le_name_mapping)  

                    df_w = pd.DataFrame(bfr)
                    df_w.to_csv(output_file_prefix+"_dir_{}_step_{}_encoded.dat".format(flow_dir,count),\
                                header=False, index=False, mode='w', sep=' ')
                count+=1
            else:
                break

            ts += STEP
    
        
    # all entries without ts
    print('[+] Generating all entries without any timeslot.')
    for flow_dir in ['0','1']:
        print('Dir :{}'.format(flow_dir))
        bfr = pd.DataFrame()
        df_entries = df_in[df_in['dir']==flow_dir][data_col]
        
        get_events_in_this(df_entries, flow_dir)
        
        df_entries.to_csv(output_file_prefix+"_dir_{}_plain.dat".format(flow_dir),\
                                header=False, index=False, mode='w', sep=' ')
        
        
        for cols in df_entries.columns.get_values():
            bfr[cols] = df_entries[cols].map(le_name_mapping)  
        
        df_w = pd.DataFrame(bfr)
        df_w.to_csv(output_file_prefix+"_dir_{}_all_encoded.dat".format(flow_dir), index=False, header=False,\
                    mode='w', sep=' ')
        
    print('[+]          Done        ')

In [3]:
def evaluate_results_wo_timeslots(i_dir, i_expanded_match = True, i_subnet = False, i_support_range=(2,100),\
                    op_file='/tmp/res.dat'):
    script_name = 'expanded_match_'
    if not i_expanded_match:
        script_name += 'exact_match_'
        
    if i_subnet:
        script_name += 'w_subnet.sh'
    else:
        script_name += 'wo_subnet.sh'
    
    script_name = "scripts/"+script_name
    
    fr = open(op_file, 'w')
    
    for i in range(i_support_range[0], i_support_range[1], 2):
        exec_string = '{} {} {}'.format(script_name, i_dir, i)
        val = subprocess.getoutput("{}".format(exec_string))
        fr.write(val+'\n')
    fr.close()

In [37]:
# 1.create test.db with table name input at i_dir.
# $sqlite3 test.db
# 2.create table input.
# CREATE TABLE input (intIP TEXT, extIP TEXT, proto TEXT, dport TEXT, dir TEXT, sport TEXT,
#                     incount INTEGER,outcount INTEGER,insize INTEGER,outsize INTEGER,scount INTEGER,
#                     pcount INTEGER, time REAL);
# 3.run this function.
def entries_to_sqlite(i_last_id, i_dir, i_change_entries = False):
    conn = sqlite3.connect(os.path.join(i_dir, 'test.db'))
    crsr = conn.cursor()
    LAST = i_last_id

    for id in range(0,LAST+1):
        df = pd.read_csv(os.path.join(i_dir, 'entries_{}.txt'.format(id)))
        df.to_sql('input', conn, if_exists='append', index=False)
    
    
    if i_change_entries:    
        # Change a few IP addresses
        # Change attack IP address 192.168.1.20 -> 137.136.135.134
        # Change DNS IP address 192.168.1.1 -> 9.9.9.9
        # Change CnC IP address 172.19.X.X -> 199.198.197.196
        quest = [
            "UPDATE input SET extIP='137.136.135.134' WHERE dport='80' and dir='0'",
            "UPDATE input SET intIP='9.9.9.9', extIP='137.136.135.134' WHERE dport='53' and dir='0'",
            "UPDATE input SET extIP='199.198.197.196' WHERE dport='8000' and dir='0'",
            "UPDATE input SET extIP='199.198.197.196' WHERE dport='4567' and not extIP like\
            '192.168.%.%' and dir='0'"
        ]

        for q in quest:
            crsr.execute(q)
            
    
    conn.commit()

    conn.close()
    print('Done........')

In [58]:
def get_bin(i_size):
    if i_size <= 100:
        return 'S'
    elif i_size > 100 and i_size <=500:
        return 'M'
    else:
        return 'L'
    
def get_all_entries(i_db, i_tbl, i_outf, i_subsampling=False):
    event_count = {}
    conn = sqlite3.connect(i_db)
    crsr = conn.cursor()
    ret_list = []
    
    # subsample login and scan outs
    if i_subsampling:
        login_attempts = "select intIP, extIP, proto, dport, dir, sport, insize/incount,\
        outsize/outcount, time from {} where dir='1' and (dport='22' and intIP like '172.16.%.%'and\
        (extIP like '172.16.%.%' or extIP like '192.168.%.%')) and pcount>0 ORDER BY RANDOM() LIMIT {}".\
        format(i_tbl, randint(1000,2000))
        
        scan_outs = "select intIP, extIP, proto, dport, dir, sport, insize/incount,\
        outsize/outcount, time from {} where dir='0' and (dport='22' and intIP like '172.16.%.%'and\
        extIP like '172.16.%.%') ORDER BY RANDOM() LIMIT {}".format(i_tbl, randint(1000,2000))
    else:    
        login_attempts = "select intIP, extIP, proto, dport, dir, sport, insize/incount,\
        outsize/outcount, time from {} where dir='1' and (dport='22' and intIP like '172.16.%.%'and\
        (extIP like '172.16.%.%' or extIP like '192.168.%.%')) and pcount>0".format(i_tbl)
    
        scan_outs = "select intIP, extIP, proto, dport, dir, sport, insize/incount,\
        outsize/outcount, time from {} where dir='0' and (dport='22' and intIP like '172.16.%.%'and\
        extIP like '172.16.%.%')".format(i_tbl)

    scan_in = "select intIP, extIP, proto, dport, dir, sport, insize/incount, outsize/outcount,\
    time from {} where dir='1' and (dport='22' and intIP like '172.16.%.%'and\
    (extIP like '172.16.%.%' or extIP like '192.168.%.%')) and pcount=0".format(i_tbl)
    
    loader = "select intIP, extIP, proto, dport, dir, sport, insize/incount, outsize/outcount,\
    time from {} where dir='0' and (dport='8000' and intIP like '172.16.%.%')".format(i_tbl)
    
    cnc = "select intIP, extIP, proto, dport, dir, sport, insize/incount, outsize/outcount,\
    time from {} where dir='0' and (dport='4567' and intIP like '172.16.%.%')".format(i_tbl)
    
    http_attack = "select intIP, extIP, proto, dport, dir, sport, insize/incount, outsize/outcount,\
    time from {} where dir='0' and (dport='80' and intIP like '172.16.%.%') ORDER BY RANDOM()\
    LIMIT {}".format(i_tbl, randint(9000,10000))
    
    dns_attack = "select intIP, extIP, proto, dport, dir, sport, insize/incount, outsize/outcount,\
    time from {} where dir='0' and (dport='53') ORDER BY RANDOM()\
    LIMIT {}".format(i_tbl, randint(9000,10000))
    
    noise = "select intIP, extIP, proto, dport, dir, sport, insize/incount, outsize/outcount,\
    time from {} where dir='1' and intIP like '172.16.%.%' and not (extIP like '172.16.%.%'\
    or extIP like '192.168.%.%')".format(i_tbl)
    
    legit = "select intIP, extIP, proto, dport, dir, sport, insize/incount, outsize/outcount,\
    time from {} where dir='0' and (dport='443' and intIP like '172.16.%.%')".format(i_tbl)

    for q,l in zip([login_attempts,scan_outs,scan_in,loader,cnc,http_attack,dns_attack, noise, legit],\
                   ['login', 'scan-out', 'scan-in', 'loader', 'cnc', 'http_ddos', 'dns_rddos', 'noise',\
                    'legit']): 
        print("{}: {}".format(l,q))
        crsr.execute(q)
        l_op = crsr.fetchall()
        if not l_op is None:
            event_count[l] = len(l_op)
            for entry in l_op:
                try:
                    insize = entry[6]
                    outsize = entry[7]
                    if entry[6] is None:
                        insize = 0
                    if entry[7] is None:
                        outsize = 0
                    size = int(insize+outsize)
                    size_bin = get_bin(size)
                    ret_list.append([entry[4], entry[0], entry[1], entry[2]+'.'+ entry[5],\
                        entry[2]+'.'+ entry[3], size_bin, entry[8], l])
                except KeyError:
                    print('Unknown direction entries in db.')
                    
    print("Total entries :{}".format(len(ret_list)))               
    df_w = pd.DataFrame(ret_list)    
    df_w.to_csv(i_outf, header=['dir', 'iot', 'ext', 'sport', 'dport',\
                                'sizeBin', 'time', 'label'], index=False)
    
    df_w = pd.read_csv(i_outf)    
    print('=======================')
    print(df_w.groupby('label').count()['iot'])

In [134]:
def get_gatewaywise_dataset(i_file, i_gw_count, op_dir='/tmp'):
    df_r = pd.read_csv(i_file, dtype=str)
    
    df_r[df_r['iot'].str.startswith('172.16.{}.'.format(i_gw_count))].\
    to_csv(os.path.join(op_dir, 'input_gateway_{}.csv'.format(i_gw_count)), index=False)
        

In [38]:
entries_to_sqlite(i_last_id =348 , i_dir = '/home/rhishi/workspace/nus/botnet_sim/manager/run_3',\
                  i_change_entries = True)

Done........


In [59]:
get_all_entries('/home/rhishi/workspace/nus/botnet_sim/manager/run_3/test.db', 'input',\
                '/tmp/test_inputs.csv', i_subsampling=True)

login: select intIP, extIP, proto, dport, dir, sport, insize/incount,        outsize/outcount, time from input where dir='1' and (dport='22' and intIP like '172.16.%.%'and        (extIP like '172.16.%.%' or extIP like '192.168.%.%')) and pcount>0 ORDER BY RANDOM() LIMIT 1524
scan-out: select intIP, extIP, proto, dport, dir, sport, insize/incount,        outsize/outcount, time from input where dir='0' and (dport='22' and intIP like '172.16.%.%'and        extIP like '172.16.%.%') ORDER BY RANDOM() LIMIT 1703
scan-in: select intIP, extIP, proto, dport, dir, sport, insize/incount, outsize/outcount,    time from input where dir='1' and (dport='22' and intIP like '172.16.%.%'and    (extIP like '172.16.%.%' or extIP like '192.168.%.%')) and pcount=0
loader: select intIP, extIP, proto, dport, dir, sport, insize/incount, outsize/outcount,    time from input where dir='0' and (dport='8000' and intIP like '172.16.%.%')
cnc: select intIP, extIP, proto, dport, dir, sport, insize/incount, outsize/ou

In [116]:
get_encoded_dataset(i_path='/tmp/test_inputs.csv', i_subnet=True, i_timeslot=300,\
                       output_mapping_file ='/tmp/test_inputs_mapping.txt',\
                       output_file_prefix = '/tmp/test_inputs')

[+] Subnet option is ENABLED.
[+] Generating timeslotted entries.
Dir :0, Timestep :0
Events->
[]
Dir :1, Timestep :0
Events->
[]
Dir :0, Timestep :1
Events->
['scan-out', 'cnc', 'loader']
Dir :1, Timestep :1
Events->
['scan-in', 'login']
Dir :0, Timestep :2
Events->
['scan-out', 'cnc', 'loader']
Dir :1, Timestep :2
Events->
['scan-in', 'login']
Dir :0, Timestep :3
Events->
['scan-out', 'cnc', 'loader']
Dir :1, Timestep :3
Events->
['scan-in', 'login']
Dir :0, Timestep :4
Events->
['scan-out', 'cnc', 'loader', 'http-ddos']
Dir :1, Timestep :4
Events->
['scan-in', 'login']
Dir :0, Timestep :5
Events->
['scan-out', 'cnc', 'http-ddos']
Dir :1, Timestep :5
Events->
['scan-in', 'login']
Dir :0, Timestep :6
Events->
['scan-out', 'cnc', 'dns-rddos']
Dir :1, Timestep :6
Events->
['scan-in', 'login']
Dir :0, Timestep :7
Events->
['scan-out', 'cnc']
Dir :1, Timestep :7
Events->
['scan-in', 'login']
Dir :0, Timestep :8
Events->
['scan-out', 'cnc']
Dir :1, Timestep :8
Events->
['scan-in', 'login']

In [137]:
for i in range(1,8):
    get_gatewaywise_dataset(i_file='/tmp/test_inputs.csv', i_gw_count=i, op_dir='/tmp')
    get_encoded_dataset(i_path='/tmp/input_gateway_{}.csv'.format(i), i_subnet=True, i_timeslot=300,\
                       output_mapping_file ='/tmp/test_inputs_mapping_gw_{}.txt'.format(i),\
                       output_file_prefix = '/tmp/test_inputs_gw_{}'.format(i))

[+] Subnet option is ENABLED.
[+] Generating timeslotted entries.
Dir :0, Timestep :0
Events->
[]
Dir :1, Timestep :0
Events->
[]
Dir :0, Timestep :1
Events->
[]
Dir :1, Timestep :1
Events->
['scan-in', 'login']
Dir :0, Timestep :2
Events->
['cnc', 'loader']
Dir :1, Timestep :2
Events->
['scan-in', 'login']
Dir :0, Timestep :3
Events->
['scan-out', 'cnc']
Dir :1, Timestep :3
Events->
['scan-in', 'login']
Dir :0, Timestep :4
Events->
['scan-out', 'cnc', 'loader', 'http-ddos']
Dir :1, Timestep :4
Events->
['scan-in', 'login']
Dir :0, Timestep :5
Events->
['scan-out', 'cnc', 'http-ddos']
Dir :1, Timestep :5
Events->
['scan-in', 'login']
Dir :0, Timestep :6
Events->
['scan-out', 'cnc']
Dir :1, Timestep :6
Events->
['scan-in', 'login']
Dir :0, Timestep :7
Events->
['scan-out', 'cnc']
Dir :1, Timestep :7
Events->
['scan-in', 'login']
Dir :0, Timestep :8
Events->
['scan-out', 'cnc']
Dir :1, Timestep :8
Events->
['scan-in', 'login']
Dir :0, Timestep :9
Events->
['scan-out', 'cnc']
Dir :1, Time

Dir :1, Timestep :11
Events->
['scan-in', 'login']
[+] Generating all entries without any timeslot.
Dir :0
Events->
['scan-out', 'cnc', 'loader', 'http-ddos']
Dir :1
Events->
['scan-in', 'login']
[+]          Done        
[+] Subnet option is ENABLED.
[+] Generating timeslotted entries.
Dir :0, Timestep :0
Events->
[]
Dir :1, Timestep :0
Events->
[]
Dir :0, Timestep :1
Events->
[]
Dir :1, Timestep :1
Events->
['scan-in', 'login']
Dir :0, Timestep :2
Events->
['scan-out', 'cnc', 'loader']
Dir :1, Timestep :2
Events->
['scan-in', 'login']
Dir :0, Timestep :3
Events->
['scan-out', 'cnc', 'loader']
Dir :1, Timestep :3
Events->
['scan-in', 'login']
Dir :0, Timestep :4
Events->
['scan-out', 'cnc', 'http-ddos']
Dir :1, Timestep :4
Events->
['scan-in', 'login']
Dir :0, Timestep :5
Events->
['scan-out', 'http-ddos']
Dir :1, Timestep :5
Events->
['scan-in', 'login']
Dir :0, Timestep :6
Events->
['scan-out']
Dir :1, Timestep :6
Events->
['scan-in', 'login']
Dir :0, Timestep :7
Events->
['scan-out